## 🚀 Tutorial: How to Create Your First AI Agent with Google Generative AI

Welcome to this comprehensive tutorial where I'll guide you step-by-step through the process of building your very own personal AI agent using the **Google Generative AI API**.

In this tutorial, we’ll cover:
- 🛠️ **Setting up your development environment** for the AI agent.
- 🔌 **Connecting to Google APIs** to integrate advanced functionalities.
- 📅 **Handling calendar data**, including fetching events and saving them to a CSV file.
- 📧 **Automating email tasks** like sending messages on your behalf.
- 🗣️ **Creating an interactive AI agent** capable of switching roles (e.g., marketing expert, supervisor, personal assistant).

---

#### 🎯 By the end of this tutorial, you’ll have a fully functional AI agent that can:
1. 📅 **Assist with calendar management**: Fetch upcoming events and save them to a CSV.
2. 📧 **Send emails automatically**: Automate email sending based on your needs.
3. 🧑‍💼 **Interact in different roles**: Use the AI agent as a marketing expert, supervisor, personal assistant, or other tailored roles.

---

Ready to get started? Let’s dive in! 🌟


In [1]:

# Install required libraries
%pip install google-generativeai pillow ipywidgets matplotlib rich secure-smtplib google-auth google-auth-oauthlib google-api-python-client


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:

# Import necessary libraries
import google.generativeai as genai
import PIL.Image
from IPython.display import display, Image, HTML
import ipywidgets as widgets
import matplotlib.pyplot as plt
import smtplib
from email.mime.text import MIMEText
from typing import List
import os
import csv
from datetime import datetime, timedelta
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from rich.markdown import Markdown
from rich.console import Console


### 🔧 Configure Google Generative AI API

Follow these steps to configure and obtain your Google Generative AI API key:

---

#### 1️⃣ Login to Your Google Account
- 🔑 **Sign in** to your Google account using [this link](https://accounts.google.com/).

---

#### 2️⃣ Navigate to Google AI Studio
- 🌐 **Go to the** [Google AI Studio](https://ai.google.dev/) page.  
  This is where you'll manage your AI projects and access API settings.

---

#### 3️⃣ Get API Key in Google AI Studio
- ⚙️ **Click on** “Explore models in Google AI Studio” and then on “Get API Key” within Google AI Studio.  
  This option will guide you through the process of setting up your API access.

---

#### 4️⃣ Create the API Key and Copy It
- 🆕 **Create a new project** if prompted.
- 📄 **Generate the API key** for your project.
- 📋 **Copy the key** and store it securely for use in your application.

---

#### ✅ Next Steps:
Now that you have your API key, you can begin integrating Google Generative AI into your application and start utilizing its capabilities!


In [ ]:
%set_env GOOGLE_GENAI_API_KEY=# TODO: Paste your API Key here 

In [5]:
genai.configure(api_key=os.environ.get("GOOGLE_GENAI_API_KEY"))

### 🔐 Authenticate Google Calendar and GMail API

Google supports **OAuth 2.0** for secure and delegated access. Follow these steps to set up and authenticate your application:

---

#### 1️⃣ Create a Google Cloud Project
- 🌐 **Go to the** [Google Cloud Console](https://console.cloud.google.com).  
- 🆕 **Create a new project** or select an existing one.  
- ✅ **Enable the Google Calendar and the Gmail API** under "APIs & Services."

---

#### 2️⃣ Configure the OAuth Consent Screen
- 🛠️ **Set up the OAuth consent screen** with your application's details.  
- 👤 **Add yourself as a Test User** for testing purposes.  
- 🔒 **Specify the required scopes**, such as:  
  ```plaintext
  https://www.googleapis.com/auth/gmail.readonly
  https://www.googleapis.com/auth/calendar.readonly

---

#### 3️⃣ Create OAuth 2.0 Credentials
- 📂 **Navigate to:**  
  `APIs & Services > Credentials` in the Google Cloud Console.  
- 🛡️ **Create Credentials:**  
  Select **OAuth 2.0 Client ID** as the credential type.  
- 💾 **Download Credentials:**  
  Save the JSON file containing your client credentials for future use.  

---

#### 4️⃣ Grant Access to Your Gmail Account
- 🔑 **Authenticate Using Client Credentials:**  
  Use the downloaded JSON file to authenticate your application and obtain an access token.  
- ✅ **Grant Permissions:**  
  During the authentication process, you'll be prompted to grant the required permissions to your AI agent.

---

#### 5️⃣ Integrate the AI Agent
- 🤖 **Use the Access Token:**  
  Utilize the authenticated token to interact with the Google Calendar API. 


In [6]:
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly', 'https://www.googleapis.com/auth/gmail.readonly']

def authenticate_google_api():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0, open_browser=False)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return build('calendar', 'v3', credentials=creds)


### 📅 Google Calendar Helper Functions

Enhance your AI agent's capabilities by seamlessly interacting with Google Calendar. These functions provide essential tools for managing calendar events, including fetching event details, saving them as CSV files, and selecting specific calendars for operations.

---

#### 🚀 Features at a Glance
- **Fetch Events**: Retrieve events from a specified Google Calendar over a chosen time range.
- **Export to CSV**: Save event details in a structured CSV format.
- **Calendar Selection**: Dynamically choose calendars for interaction.

---

#### 🎯 Use Case
These functions are perfect for integrating Google Calendar event management into larger applications, enabling:
- **Automated Workflows**: Export and process calendar events effortlessly.
- **Event Analysis**: Structure data for analytics or reporting.
- **Cross-Platform Integration**: Synchronize event data across multiple systems.

With these tools, your application can efficiently handle Google Calendar data and provide meaningful insights to users.


In [7]:

# Fetch calendar events from Google Calendar
def fetch_calendar_events(service, days=7, calendar_id='primary'):
    """
    Fetches events from a specified Google Calendar for a given period.

    Parameters:
        service: Authenticated Google Calendar API service object.
        days (int): Number of days ahead to fetch events (default: 7).
        calendar_id (str): ID of the calendar to fetch events from (default: 'primary').

    Returns:
        list: A list of event objects retrieved from the calendar.
    """
    now = datetime.utcnow().isoformat() + 'Z'
    future = (datetime.utcnow() + timedelta(days=days)).isoformat() + 'Z'
    events_result = service.events().list(
        calendarId=calendar_id, timeMin=now, timeMax=future,
        singleEvents=True, orderBy='startTime').execute()
    events = events_result.get('items', [])
    return events


# Write fetched calendar events to CSV
def write_events_to_csv(events, output_csv):
    """
    Writes a list of Google Calendar events to a CSV file.

    Parameters:
        events (list): A list of event objects fetched from Google Calendar.
        output_csv (str): Path to the output CSV file.

    CSV Fields:
        - Summary: Event title.
        - Start: Start time or date of the event.
        - End: End time or date of the event.
        - Location: Event location (if available).
        - Description: Event description (if available).
    """
    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Summary', 'Start', 'End', 'Location', 'Description']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for event in events:
            writer.writerow({
                'Summary': event.get('summary', 'No Title'),
                'Start': event['start'].get('dateTime', event['start'].get('date')),
                'End': event['end'].get('dateTime', event['end'].get('date')),
                'Location': event.get('location', 'N/A'),
                'Description': event.get('description', 'N/A'),
            })

# Select a calendar
def select_calendar(service):
    """
    Prompts the user to select a calendar from a list of available calendars.

    Parameters:
        service: Authenticated Google Calendar API service object.

    Returns:
        str: The ID of the selected calendar.
    """    
    calendars = service.calendarList().list().execute()
    print("Available Calendars:")
    for i, calendar in enumerate(calendars['items']):
        print(f"{i + 1}: {calendar['summary']} (ID: {calendar['id']})")

    choice = int(input("Enter the number of the calendar you want to use: ")) - 1
    return calendars['items'][choice]['id']

# Write Google Calendar events to a CSV file
def write_google_calendar_to_csv(days: int, output_csv: str): # doesn't work with predefined values e.g. days=7
    """
    Combines multiple functions to:
    1. Authenticate the Google API.
    2. Allow the user to select a calendar.
    3. Fetch events for the specified duration.
    4. Save the events to a CSV file.

    Parameters:
        days (int): Number of days ahead to fetch events.
        output_csv (str): Path to save the events as a CSV file.
    """
    service = authenticate_google_api()
    calendar_id = select_calendar(service)  # Let the user choose a calendar
    events = fetch_calendar_events(service, days=days, calendar_id=calendar_id)
    # events = fetch_calendar_events(service, days=7)

    if not events:
        print("No upcoming events found.")
        return

    # Write events to a CSV file
    write_events_to_csv(events, output_csv)

In [8]:
def save_text_to_file(text: str, filename: str):
    """
    Saves the given text to a file with a maximum of 80 words per line.
    
    Parameters:
        text (str): The text to be saved.
        filename (str): The name of the file to save the text in.
    """
    text = text.replace("\\n", "\n")

    # Split the text into words
    words = text.split()
    
    # Open the file for writing
    with open(filename, 'w') as file:
        # Loop through the words in chunks of 20
        for i in range(0, len(words), 20):
            # Join up to 20 words into a line
            line = ' '.join(words[i:i+20])
            file.write(line + '\n')

### 🔑 Give Access to GMAIL: App Password Setup

To allow your AI agent to access your Gmail account securely, you'll need to create an **App Password**. Follow these steps:

---

#### 1️⃣ **Sign in to Your Google Account**
- 🌐 Visit [Google Account Sign In](https://accounts.google.com) and log in.

---

#### 2️⃣ **Navigate to Security Settings**
- 🔒 In the **Security** section, find **"Signing in to Google"**.
- 🛡️ Click on **2-Step Verification** > **App Passwords** (you may be prompted to sign in again).

---

#### 3️⃣ **Check Eligibility**
If you don’t see the **App Passwords** option, it might be because:
- 🚫 **2-Step Verification** is not enabled for your account.
- 🔑 **2-Step Verification** is set up for **security keys** only.
- 🏢 Your account is managed by your **work, school, or other organization**.
- 🛡️ You have **Advanced Protection** enabled.

---

#### 4️⃣ **Generate the App Password**
- 📱 Click **Select app** and choose the app you're generating the password for.
- 🖥️ Click **Select device** and pick the device you're using.
- 🔑 Click **Generate** to create the app password.

---

#### 5️⃣ **Enter the App Password**
- ✍️ Follow the instructions to enter the **16-character app password** shown in the yellow bar on your device.
- ✅ Tap **Done** to complete the process.

---

#### 💡 **Tip:**
- 🔄 You’ll only need to enter the app password **once per app or device**, so you won’t need to memorize it for future use.

---

By following these steps, you'll securely authorize your AI agent to interact with Gmail. 🛠️


In [ ]:
%set_env GMAIL_PASSWD=# TODO: Paste your password here 

In [10]:
def send_email(subject: str, body: str, sender: str, recipients: List[str]):
   """
    Sends an email with the specified subject, body, sender, and recipients.

    Parameters:
    - subject (str): The subject of the email.
    - body (str): The content of the email body, where "\\n" is replaced with actual line breaks.
    - sender (str): The email address of the sender.
    - recipients (List[str]): A list of recipient email addresses.

    Process:
    - The email subject is set with an optional "[labcamp-tutorial]" suffix.
    - The body text is formatted, replacing escaped newline characters ("\\n") with actual newlines.
    - The function retrieves the sender's email password from environment variables for secure login.
    - Uses Gmail's SMTP server to send the email securely via SSL.
    - Prints a confirmation message after the email is sent.

    Note:
    - Ensure that the `GMAIL_PASSWD` environment variable is set with the correct Gmail account password.
    """
    
   formatted_body = body.replace("\\n", "\n")
   msg = MIMEText(formatted_body)
   msg['Subject'] = subject + " [labcamp-tutorial]"
   msg['From'] = sender
   msg['To'] = ', '.join(recipients)
   password = os.environ["GMAIL_PASSWD"]
    
   with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp_server:
      smtp_server.login(sender, password)
      smtp_server.sendmail(sender, recipients, msg.as_string())
   print("Message sent!")

### 🤖 Define Roles and Chatbot Interaction Logic

In the following we dive into the AI Agent logic. 
We initialize the **Google Gemini 2.0** generative model and equip it with several tools to **send Emails**, **access Google calendars**, and **save generated text to files**. The model is then used to start a chatbot interaction with automatic function calling enabled.

A set of predefined roles is created, including:
- **Supervisor**: Provides guidance and feedback to improve products.
- **Marketing**: Focuses on creating effective campaigns and strategies.
- **Assistant**: Assists with tasks like saving text, managing calendars, and sending emails.

The **current role** is set to **marketing**, making it the default role for the chatbot interaction.


In [11]:
# Define roles and chatbot interaction logic

model = genai.GenerativeModel(model_name='gemini-2.0-flash-exp', tools=[send_email, write_google_calendar_to_csv, save_text_to_file])
print(model)

chat = model.start_chat(enable_automatic_function_calling=True)

roles = {
    "supervisor": "You are a supervisor. Your role is to provide guidance and constructive feedback to improve products.",
    "marketing": "You are a marketing expert. Your role is to create effective campaigns and marketing strategies.",
    "assistant": "You are a personal assistant. You help with saving text to files, managing calendars, and sending emails.",
}

current_role = "marketing"  # Default role

genai.GenerativeModel(
    model_name='models/gemini-2.0-flash-exp',
    generation_config={},
    safety_settings={},
    tools=<google.generativeai.types.content_types.FunctionLibrary object at 0x000002571C965950>,
    system_instruction=None,
    cached_content=None
)


In [12]:

def parse_command(user_input): 
    """
    Parses the user input to identify if it contains a command starting with a '/'.
    
    Args:
        user_input (str): The input provided by the user.
        
    Returns:
        tuple: A tuple (role, text), where:
            - role (str): The command's role extracted from the input (if it starts with '/').
            - text (str): The remaining text after the command.
        If the input does not start with '/', returns (None, user_input).
    """
    if user_input.startswith("/"):
        command, *rest = user_input.split(" ", 1)  # Split input into command and rest
        role = command[1:]  # Remove the leading '/' from the command
        text = rest[0] if rest else ""  # Get the rest of the input or an empty string
        return role, text
    return None, user_input  # Return None for role if no command is detected


def generate_full_prompt(user_input, current_role):
    """
    Constructs a full prompt for the chatbot by combining a role-specific instruction with the user's input.
    
    Args:
        user_input (str): The input provided by the user.
        current_role (str): The current role of the chatbot (used to fetch role-specific instructions).
        
    Returns:
        str: A formatted string containing the role instruction, user input, and chatbot label.
    """
    role_instruction = roles.get(current_role, "")  # Fetch the instruction for the current role
    full_prompt = f"{role_instruction}\n\nUser: {user_input}\nChatbot:"  # Format the full prompt
    
    return full_prompt



### 🔄 Explanation of `chatbot_loop` Function

The `chatbot_loop` function manages the interaction between the user and the chatbot, allowing role-based functionality and file uploads. Here's what happens step-by-step:

---

1. **Initialization**:
   - A **global `current_role`** variable is used to track the active role for the chatbot.
   - A **console** is created for rich-text output.
   - Instructions are printed to guide the user on available commands (`/role`, `/upload`).

---

2. **Main Loop**:
   - The chatbot continuously waits for user input until the user types an **exit command** like `exit`, `quit`, or `thanks`.

---

3. **File Upload Handling**:
   - The user can upload a file with the `/upload <file_path> /<role> <prompt>` command.
   - If the file exists, it is uploaded using `genai.upload_file` with metadata (e.g., file name and MIME type).
   - If the file doesn't exist or the command is improperly formatted, an error message is displayed.

---



In [13]:

def chatbot_loop():
    global current_role
    console = Console()
    print("Type /role <query> to switch roles. Available roles: supervisor, marketing agent, assistant.")
    print("Type /upload <file_path> /<role> <prompt> to upload a file and provide a role-specific query.")
    print("Available roles: supervisor, marketing agent, assistant.")
    
    while True:
        user_input = input("You: ").strip()
        
        # Check for exit command
        if user_input.lower() in {"exit", "quit", "thanks"}:
            print("Chatbot: Goodbye!")
            break
        
        file_path = None  # Reset file_path for each input

        # Check for combined /upload and /<role> command
        if user_input.startswith("/upload "):
            try:
                # Extract file path and command/query part
                file_part, command_part = user_input[8:].split("/", 1)
                file_path = file_part.strip()
                command_part = f"/{command_part.strip()}"  # Add back the slash to the command
                
                if os.path.isfile(file_path):
                    file_path = genai.upload_file(path=file_path)
                    print(f"Chatbot: File '{file_path}' uploaded successfully.")
                else:
                    print(f"Chatbot: File '{file_path}' not found.")
                    continue

                # Use the command_part (with the dynamic command) for further processing
                user_input = command_part
            except ValueError:
                print("Chatbot: Invalid command format. Use /upload <file_path> /<role> <prompt>.")
                continue

        # Parse user input for role and query
        role, query = parse_command(user_input)
        if role:
            if role in roles:
                current_role = role
                print(f"Chatbot: Role switched to {role}.")
                if query:
                    # Generate the full prompt
                    full_prompt = generate_full_prompt(query, current_role)
                    print(full_prompt)
                    
                    # Adjust arguments based on whether file_path is provided
                    inputs = [full_prompt, file_path] if file_path else [full_prompt]
                    print(inputs)
                    
                    # Send the message to the chat system
                    response = chat.send_message(
                        inputs,
                        request_options={"timeout": 900}
                    )
                    console.print(Markdown(response.text))
            else:
                print(f"Chatbot: Unknown role '{role}'. Please choose from: {', '.join(roles.keys())}.")
        else:
            print("Chatbot: Could not parse the role or query.")


In [ ]:

def chatbot_loop():
    global current_role
    console = Console()
    print("Type /role <query> to switch roles. Available roles: supervisor, marketing agent, assistant.")
    print("Type /upload <file_path> /<role> <prompt> to upload a file and provide a role-specific query.")
    print("Available roles: supervisor, marketing agent, assistant.")
    
    while True:
        user_input = input("You: ").strip()
        
        # Check for exit command
        if user_input.lower() in {"exit", "quit", "thanks"}:
            print("Chatbot: Goodbye!")
            break
        
        # TODO: Reset file_path for each input

        # TODO: Handle combined /upload and /<role> command
        # Example: Extract file_path and command_part
        # TODO: Check if the file exists and upload it using genai.upload_file
        # TODO: Set user_input to command_part for further processing

        # TODO: Parse user input to extract the role and query

        # TODO: Handle role switching logic
        # Example: Switch to a new role if valid, and generate the prompt if a query is provided

        # TODO: Generate full prompt and send a message to the chat system
        # Example: Use the chat.send_message method to send inputs and print the response

        # TODO: Handle the case where the role is unknown or the command cannot be parsed

In [ ]:
img = PIL.Image.open('bottle.jpg')
plt.imshow(img)
plt.show()

analyzePrompt = """This image contains a sketch of a potential product along with some notes.
Given the product sketch, describe the product as thoroughly as possible based on what you
see in the image, making sure to note all of the product features.
"""

print(analyzePrompt)

response = chat.send_message(
        [analyzePrompt, img]
)

# Render Markdown in the terminal
console = Console()
console.print(Markdown(response.text))

chatbot_loop()